In [ ]:
import polars as pl
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
from synthetic_data.tasks.writing import GutenbergBacktranslationFromTxt
gutenberg_pq: pl.DataFrame = pl.read_parquet("../dataset_files/epubs.parquet")

task = GutenbergBacktranslationFromTxt()

dataset = gutenberg_pq
load_dotenv("../.env")

In [ ]:
task

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset.to_pandas())
sample_out = await task.preprocess_row(dataset[4])
sample_out = sample_out[0:8]
sample_out

In [31]:
from synthetic_data.generation import get_generation_wrapper


generation_wrapper = get_generation_wrapper("gemini-2.0-flash")
out = await task.generate(generation_wrapper, sample_out)
out

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-03-21 21:27:11.838 | INFO     | synthetic_data.tasks.writing:generate:417 - [1, 1, 1, 1, 1, 1, 1, 1]
2025-03-21 21:27:11.839 | INFO     | synthetic_data.tasks.writing:generate:422 - Valid rows: 8


[{'completion': '<analysis>\na) **Brief Summary:** This paragraph introduces the protagonist, Alerik, in a dystopian Martian settlement, as he reflects on a conversation with Velar about the truth behind their society\'s origins and their distrust of "The Union." He then seeks solace in a cheap drug while contemplating the oppressive nature of their world.\n\nb) **Writing Style:** The writing style is casual and descriptive, using fairly simple language to paint a picture of the setting and characters. Sentences like, "They were really creative with names," and "Shit was fucked up," exemplify the casual tone. The descriptive elements include details about Alerik\'s appearance and the bar.\n\nc) **Tone:** The tone is cynical and weary, reflecting the protagonist\'s disillusionment with the dystopian society. Phrases such as "distilled shit they were passing as a drink" and the description of The Union\'s empty promises contribute to this tone. There\'s also an underlying sense of tensio

In [30]:
from synthetic_data.prompts import tags_to_instruction


print(tags_to_instruction(out[0]['tags']))


You are an expert writer. Your task is to take the following tags and create a detailed piece of writing.
- Write in a cynical and world-weary tone. Convey disillusionment and resignation, while hinting at protectiveness toward another character. Use a detached narrative voice that effectively expresses internal thoughts. Aim for a bleak and oppressive atmosphere reflecting a dystopian setting.
- The paragraph should open with the protagonist reflecting on a recent conversation with a younger or more naive individual. This reflection should reveal the protagonist's dissatisfaction with the current social or political situation. The protagonist acknowledges a truth they can't openly share for fear of consequences and actively hides this from the other person to protect them. The protagonist then engages in a solitary activity, like drinking at a bar or smoking, to cope with their disillusionment.
- Begin with an internal monologue reflecting on the past conversation. Follow with a desc

In [ ]:
print('hi')
for row in out:
    print(row)
    print(row.keys())
    print(row['tags'])
    print()


In [ ]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from synthetic_data.tasks.writing import find_valid_paragraph_chunks
from IPython.display import display, Markdown

tiktoken_encoder = tiktoken.get_encoding("o200k_base")

df_sampled = df.sample(100)
for row in df.iter_rows(named=True):
    text = row["text"]
    paragraphs = super_cleaner(text)

    # find chunks of consecutive paragraphs with dialogue and at least 3 sentences
    valid_chunks = find_valid_paragraph_chunks(paragraphs, tiktoken_encoder)
    display(Markdown(f"### {row['title']}"))
    for chunk in valid_chunks:
        display(Markdown(chunk[0]))

